In [ ]:
from dependencies import *
import torch


In [3]:
checkpoint_quick_test= torch.load("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/checkpoints/model_checkpoint_UNet_01.pth")

In [5]:
#Checking the parameters and keys
print(type(checkpoint_quick_test))

<class 'dict'>


In [8]:
#Checking all model parameters 
for key in checkpoint_quick_test.keys():
    print(key)

epoch
model_state_dict
optimizer_state_dict
loss


In [9]:
#Importing UNet class from Unet.py

In [10]:
import sys
import os
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/ML_models")

In [15]:
from UNet import UNet
from Downscaling_Dataset_Prep import DownscalingDataset
from Downscaling_Dataset_Prep import PairedDataset

In [16]:
model_instance= UNet(in_channels=2, out_channels=2)
model_instance.load_state_dict(checkpoint_quick_test["model_state_dict"])
model_instance.eval()

UNet(
  (Encoder1): Encoder_Block(
    (conv): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (Encoder2): Encoder_Block(
    (conv): DoubleConv(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), b

In [ ]:
#Scaled datasets for testing :loading


In [19]:
precip_input = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_precip_masked_bicubic_test.nc")
temp_input = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_tas_masked_bicubic_test.nc")
precip_target = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_precip_masked_test.nc")
temp_target= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_tas_masked_test.nc")

In [23]:
#Building the paired feature-target dataset; first loading individual and coverting them into images

In [41]:
precip_ds= DownscalingDataset(precip_input, precip_target, var_name_inputs="pr", var_name_targets="RhiresD")
temp_ds = DownscalingDataset(temp_input, temp_target, var_name_inputs='tas', var_name_targets='TabsD')


In [45]:
from torch.utils.data import DataLoader

In [46]:
#Now building paired dataset
paired_ds= PairedDataset(precip_ds,temp_ds)
test_loader= DataLoader(paired_ds, batch_size=32, shuffle=False)

Predictions

In [ ]:
predictions = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model_instance(inputs)
        predictions.append(outputs)

#Stacking predictions
predictions = torch.cat(predictions, dim=0) 

print(f"Final Predictions shape: {predictions.shape}")
